# Physics 453: Finite Spherical Well

You've learned about how to solve the Schrodinger equation in 3 dimensions for spherically symmetric potentials, $V(\vec{r})=V(r)$, by using spherical coordinates and searching for a separable solution. As shown in Chapter 4 of Griffiths, the angular part of the solution does not depend on the details of the solutions, and is given by one of the *spherical harmonics*.  The real problem is to solve the *radial equation*, which when we switch variables to $u(r)=rR(r)$ is given by:
$$-\frac{\hbar^2}{2m} \frac{d^2u}{dr^2}+\left[ V(r)+\frac{\hbar^2}{2m}\frac{l(l+1)}{r^2}\right]u=Eu$$

Where $k=\sqrt{(E-V)2m}/\hbar$. As noted by Equation [4.45] in Griffiths, the general solution to this equation when $V(R)=V$ is constant is
$$ u(r)=Arj_l(kr)+Brn_l(kr)$$
where $j_l(x)$ is the spherical Bessel function (of the first kind) of order $l$ and $n_l(x)$ is the spherical Neumann function (or Bessel function of the second kind) of order $l$. When we are looking for the solution is a region of space where the wavefunction should decay (ie, $E<V$), then we should use Hankel functions, which are just linear combinations of the Bessel and Neumann functions. See the first Reference for more about Hankel functions.

### References
More about choosing the right special functions for each region: http://quantummechanics.ucsd.edu/ph130a/130_notes/node225.html

How to get Bessel, Hankel, etc. functions (and their derivatives!) in Python: https://docs.scipy.org/doc/scipy/reference/special.html

How to use brentq from SciPy: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.brentq.html

### Finite Spherical Well
In this notebook you are going to find the bound state solutions to the finite spherical well for arbitrary $l$.  The potential is given by $V(r)=-|V|$ for $r<=a$, and $V(r)=0$ for $r>a$.  Applying the wavefunction boundary conditions at $r=a$ will result in a transcendental equation, which we need to solve numerically.

We will use the following parameters:
$$\hbar=m=1$$
$$a=2$$
$$V=-10$$

#### Question 1
The first step is to derive the transcendental equation the arises from the boundary conditions for the finite spherical well.  Do this on paper and write the transcendental equation you found in the box below. (You don't have to hand in your written work.)

*Write your result here!*

Now let's get to coding.  First we'll load the libraries we'll need.

In [ ]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt 
import numpy
import scipy.special    #Contains the bessel, neumann, and hankel function methods
import scipy.optimize   #Contains brentq, the root finding algorithm we'll use

In [ ]:
hbar=1
m=1
a=2
V=-10

#### Question 2

Let's get comfortable with using the special function methods from scipy.  Make a plot of the $l=0,1,2$ Bessel functions (of the first kind) *and* their derivatives over $x=0..10$. (Hint: Take a look at the second reference.)

In [ ]:
#Make a plot!


#### Question 3
The transcendental equation you derived should have the form:
$$f_l(E) = g_l(E)$$
That is, some function of the energy on both sides, with $l$ as a parameter. The energies, $E$, that satisfy this equation (for the given angular momentum $l$) are the bound state energies.  Alternatively, we can form a new function:
$$Q_l(E) = f_l(E)-g_l(E)$$
Now the energies that we seek are the roots of $Q_l(E)$, ie. $Q_l(E)=0$.

Write a function, called *boundary_function*, that takes $E$ and $l$ as arguments and returns the value of $Q_l(E)$. (You may also want to write short helper functions that calculate the value of $k$ in either region.)

Then make a plot of $Q_{l=0}(E)$ for $E=-|V|..0$.  Does $Q_l$ have an imaginary part?  How many bound states are there for $l=0$, and what are their approximate energies? Note: you may get some error messages below the plot about dividing by zero.  Don't worry about them, they are just the result of trying to plot where the curve blows up to positive or negative infinity.

In [ ]:
#Fix these two functions
def k1(E):
    return E

def k2(E):
    return E

#Getting the hankel functions from SciPy is a bit complex, so I've provided these functions to calculate
#the hankel function and it's first derivative.  However, you can get the bessel functions (and their derivatives) easily
#from the SciPy library
def hankel(l, z):
    return numpy.sqrt(numpy.pi/(2*z))*scipy.special.h1vp(l+0.5,z,0)
    
def hankel_deriv(l, z):
    return numpy.sqrt(numpy.pi/(2*z))*scipy.special.h1vp(l+0.5,z,1) - 0.5*(1./z)*numpy.sqrt(numpy.pi/(2*z))*scipy.special.h1vp(l+0.5,z,0)
  
#transcendental equation arising from matching boundary conditions for finite spherical well
def boundary_function(E, l):
    
    left = 2 #What is the left side of the transcendental equation?
    right = 3 #What is the right side?
    
    return (left-right).real


#Make a plot
#Plot the transcendental equation arising from the boundary conditions
#Zeroes of this function are physical energy solutions

*Write the written part of the answer here*

#### Question 4
You should have seen that $Q_l(E)$ is a purely real function, so modify your function to return only the real part. (Note: probably you'll see a very small imaginary part; this is due to numerical error, and is why we need to explicitly return the real part.)

You can eyeball the energies from the plot, but we want to do better than that.  We'll start off by writing a function that takes a range of energies and return a list of pairs of energies, $(E-dE, E)$, between which there is a zero of $Q_l(E)$ (or equivalently, between which $Q_l(E)$ changes sign).  Do the results for $l=0$ seem to agree with your plot?

In [ ]:
#Implement this function!
def acquire_zero_bounds(energy_range, l): 
    zeros_list=[]  #Add energy pairs to list list if we think there is a zero in between them
    value = boundary_function(energy_range[0],l)  #Get the starting value of boundary_function
    
    #Write some code to find the pairs of energies between which there is a zero - this is the only part of this code cell you need to change
    
    return zeros_list

#Sweep through energies in steps on dE, starting from the lowest possible energy (V=-|V|) to the highest (0)
dE = abs(V)/1000
energy_range = numpy.arange(V, 0, dE)

#Finding solutions of this angular momentum quantum number, l
l=0

energy_bounds = acquire_zero_bounds(energy_range, l)
print(energy_bounds)


#### Question 5
Now we know the correct energies to within a small $\Delta E$. Now we can use a very nice root finding algorithm, called *brentq*, to find the zero to a very high degree of accuracy.  All the algorithm needs is the function, any extra arguments that it takes, and a sign-changing interval (which we just found!)

Use SciPy's brentq method to find the (numerically) exact bound state energies. (You should look at the online documentation for brentq!)

In [ ]:
def acquire_energies(energy_bounds, l):
    energies = []
    for energy_bound in energy_bounds:
        energies.append( scipy.optimize.brentq() ) #What should the arguments of brentq be?
    return energies

energies = acquire_energies(energy_bounds, l)

#### Question 6
Now that we have the right energies, we need to construct the wavefunctions.  Write a function that takes the energies, a list of radii (positions), and the value of $l$ and returns a list of the wavefunctions for the possible bound states.  Also write a function that properly normalizes the wavefunction. (Remember that this is a radial wavefunction is spherical coordinates, so the norm looks a bit diferent!)  

In [ ]:
#Plot each solution found for the given l
dr=0.01
max_r = 6
space = numpy.arange(0, max_r, dr)
inner = space <= a
outer = space > a

#Normalize the radial part of the wavefunction - Remember the r^2!
def normalize(psi, space, dr):
    norm = 1 #What should go here?
    return psi / numpy.sqrt(norm)

def get_wavefunctions(space, energies, l):
    psis=[]
    for energy in energies:
        psi = numpy.zeros(len(space), complex)
        #How do we calcualte psi?
        psi[outer] = space[outer]   #Examples of array masking, but you'll need to replace these examples!
        psi[inner] = space[inner]
        
        psis.append(normalize(psi,space,dr))
        
    return psis

#### Question 7
This code will make a plot of the radial probability distribution, $RPD$, for all bound states for $l=0$.  It then creates an energy level diagram for all bound state energies for $l=0,1,2$, and color codes the lines based on $l$.
$$RPD = |R|^2r^2$$
For this choice of well parameters, what is the value of $l$ for the ground state and the highest energy bound state?

In [ ]:
# Get all the bound state wavefunctions for l=0 and plot them
plt.figure()

wavefunctions = get_wavefunctions(space, energies, l)
for i in range(0,len(wavefunctions)):
    plt.plot(space, (numpy.conjugate(wavefunctions[i])*wavefunctions[i]*space*space).real, label="E="+str("%.2f" % energies[i]))
    
plt.xlabel('r')
plt.ylabel(r'$|R|^2r^2$')
plt.title('Bound States of Finite Spherical Well for l='+str(l))
plt.legend()
plt.show()

#Make an energy level diagram for l=0,1,2.  I've already started by drawing the potential, just draw the energy levels on top of it.
plt.figure()
plt.plot([0,a],[V,V], 'k', label="Potential")
plt.plot([a,a],[V,0], 'k')
plt.plot([a,max_r],[0,0], 'k')

colors = ['blue','red', 'green', 'purple', 'orange']
for l_new in [0,1,2]:
    bounds_new = acquire_zero_bounds(energy_range, l_new)
    energies_new = acquire_energies(bounds_new, l_new)
    for energy in energies_new:
        plt.plot([0,a],[energy,energy], color=colors[l_new], lw=3, label="l="+str(l_new))

plt.legend()
plt.title('Energy Level Diagram for Finite Spherical Well')
plt.xlabel('r')
plt.ylabel('Energy')
plt.ylim([-12,5])
plt.show()

Write answer to question 7 here.